<a href="https://www.kaggle.com/code/bharathkumar1011/resume-parsing-using-spacy?scriptVersionId=217944240" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install -qU langchain_community
!pip install pdfplumber

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 60.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 63.4 MB/s eta 0:00:00:00:01


In [2]:
import spacy
import re
import pandas as pd
from langchain_community.document_loaders import PDFPlumberLoader
from spacy.matcher import Matcher

In [4]:
def extract_text_pdf(pdf_path):
        loader = PDFPlumberLoader(pdf_path)
        docs = loader.load()
        text = ''
        for doc in docs:
            text += doc.page_content # Append extracted text
        return text 


def extract_name(nlp_text, matcher):
    '''
    Helper function to extract name from spacy nlp text

    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param matcher: object of `spacy.matcher.Matcher`
    :return: string of full name
    '''
    pattern =  [{'POS': 'PROPN'}, {'POS': 'PROPN'}] 
    """extrcted particular pattern variable
        rather than imporing from a large data
        """
    
    matcher.add("NAME", [pattern])# changed from "matcher.add('NAME', None, *pattern)"
    
    matches = matcher(nlp_text)
    
    for match_id, start, end in matches:
        span = nlp_text[start:end]
        return span.text

def extract_email(resume_text):
    '''
    Helper function to extract email id from text

    :param text: plain text extracted from resume file
    '''
    email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)", resume_text)
    if email:
        try:
            return email[0].split()[0].strip(';')
        except IndexError:
            return None



def extract_education_spacy(resume_text):
    """
    Extracts the Education section from resume text using spaCy.
    
    :param resume_text: Plain resume text
    :return: Extracted "Education" section as a string
    """

    # Split the resume into lines
    lines = resume_text.split("\n")

    education_section = []
    
    # Extract lines after "EDUCATION" until the next section
    for line in lines:
        if education_section and re.search(r"\b(project|skills|work experience|certifications)\b", line, re.IGNORECASE):  
            break 
    
        if re.search(r"\b(education)\b", line, re.IGNORECASE) or education_section:  
            education_section.append(line.strip())
    
    education_text = "\n".join(education_section)

    education_text = education_text.replace('\\n', ' ').replace('\n', ' ').strip()

    return education_text




def extract_skills(nlp_text, noun_chunks):
    '''
    Helper function to extract skills from spacy nlp text

    :param nlp_text: object of `spacy.tokens.doc.Doc`
    :param noun_chunks: noun chunks extracted from nlp text
    :return: list of skills extracted
    '''
    tokens = [token.text for token in nlp_text if not token.is_stop]
    
    # if noun_chunks is None:
    #     noun_chunks = list(nlp_text.noun_chunks)

    
    url = "https://raw.githubusercontent.com/OmkarPathak/ResumeParser/refs/heads/master/resume_parser/resume_parser/skills.csv"
    df = pd.read_csv(url, index_col=0)
    data = df
    
    skills = list(data.columns.values)
    
    skillset = []
    # check for one-grams
    for token in tokens:
        if token.lower() in skills:
            skillset.append(token)
    
    # check for bi-grams and tri-grams
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in skills:
            skillset.append(token)
    return [i.capitalize() for i in set([i.lower() for i in skillset])]


def extract_experience_spacy(resume_text):
    '''
    Extracts the experience section from resume text using spaCy.
    Also extracts job titles, company names, and dates.
    
    :param resume_text: Plain resume text
    :return: Extracted "Experience" section as a string
    '''


    # Step 1: Extract the "Experience" section based on headers
    lines = resume_text.split("\n")
    experience_section = []
    capture = False

   # Step 1: Start capturing after "WORK EXPERIENCE" or similar headers
    for line in lines:
        if re.search(r"\b(experience|work experience|employment history)\b", line, re.IGNORECASE):  # Detect section start
            capture = True
        elif capture and re.search(r"\b(education|skills|projects)\b", line, re.IGNORECASE):  # Stop at next section
            break
        if capture:
            experience_section.append(line.strip())

    experience_text = "\n".join(experience_section)
    experience_text = experience_text.replace('\\n', ' ').replace('\n', ' ').strip()

    # Step 2: Extract structured entities using spaCy
    experience_doc = nlp(experience_text)
    extracted_info = {
        "Job Titles": [],
        "Companies": [],
        "Dates": []
    }
    return experience_text
    


In [5]:
pdf_path = "/kaggle/input/barath-resume/Bharath_Kumar_Resume.pdf"

In [6]:
text = extract_text_pdf(pdf_path)

In [7]:
nlp = spacy.load("en_core_web_sm")
nlp_text = nlp(text)
noun_chunks = list(nlp_text.noun_chunks)
matcher = Matcher(nlp.vocab)


In [8]:
output = {
    "Name": extract_name(nlp_text, matcher),
    "Email": extract_email(text),
    "Skills":extract_skills(nlp_text, noun_chunks),
    "Education": extract_education_spacy(text),
    "Experience": extract_experience_spacy(text),
}

In [10]:
print(output)

{'Name': 'Bharath Kumar', 'Email': 'bharathkumar1011@gmail.com', 'Skills': ['Modeling', 'Python', 'Analysis', 'Analyze', 'Engineering', 'Communication', 'Sql', 'Data quality', 'Data analysis', 'Programming', 'Jupyter', 'Statistical analysis', 'Analytics', 'Technical', 'Reports', 'Database', 'Pattern', 'Datasets', 'Etl', 'Excel'], 'Education': 'Education B.Tech in Aeronautical Engineering Institute of Aeronautical Engineering 2014 - 2018 GPA: 6.18/10', 'Experience': 'Results-driven Data Scientist with two years of experience in data analysis, predictive modeling, and data visualization. Passionate about leveraging data-driven insights to optimize business strategies and enhance decision-making processes. Experience Data Scientist XYZ Company, Hyderabad January 2022 - Present - Developed predictive models to analyze customer behavior and optimize business strategies. - Performed exploratory data analysis (EDA) and data cleaning to improve data quality. - Designed and maintained ETL pipel

In [11]:
for k,v in output.items():
    print(k,v)

Name Bharath Kumar
Email bharathkumar1011@gmail.com
Skills ['Modeling', 'Python', 'Analysis', 'Analyze', 'Engineering', 'Communication', 'Sql', 'Data quality', 'Data analysis', 'Programming', 'Jupyter', 'Statistical analysis', 'Analytics', 'Technical', 'Reports', 'Database', 'Pattern', 'Datasets', 'Etl', 'Excel']
Education Education B.Tech in Aeronautical Engineering Institute of Aeronautical Engineering 2014 - 2018 GPA: 6.18/10
Experience Results-driven Data Scientist with two years of experience in data analysis, predictive modeling, and data visualization. Passionate about leveraging data-driven insights to optimize business strategies and enhance decision-making processes. Experience Data Scientist XYZ Company, Hyderabad January 2022 - Present - Developed predictive models to analyze customer behavior and optimize business strategies. - Performed exploratory data analysis (EDA) and data cleaning to improve data quality. - Designed and maintained ETL pipelines to streamline data pro